# Carregando os Pacotes

In [1]:
%env TF_CPP_MIN_LOG_LEVEL = 3

env: TF_CPP_MIN_LOG_LEVEL=3


In [2]:
import torch
import gradio as gr
from transformers import AutoModelForCausalLM
import warnings
warnings.filterwarnings('ignore')

/home/renan/anaconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carregando o LLM Treinado

In [3]:
modelo_llm = AutoModelForCausalLM.from_pretrained('llm/llm_final/')

In [4]:
modelo_llm

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(13, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=13, bias=False)
)

# Carregando o Tokenizador

In [19]:
class Tokenizador:

    def __init__(self, numbers_qty = 10):

        vocab = ['+', '=', '-1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        self.numbers_qty = numbers_qty
        self.pad_token = '-1'
        self.encoder = {str(v):i for i, v in enumerate(vocab)}
        self.decoder = {i:str(v) for i, v in enumerate(vocab)}
        self.pad_token_id = self.encoder[self.pad_token]

    def decode(self, token_ids):
        return ' '.join(self.decoder[t] for t in token_ids)

    def __call__(self, text):
        return [self.encoder[t] for t in text.split()]

In [20]:
tokenizador = Tokenizador(13)

In [21]:
tokenizador.encoder

{'+': 0,
 '=': 1,
 '-1': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12}

In [22]:
tokenizador.decoder

{0: '+',
 1: '=',
 2: '-1',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Criando Função Para Previsões com o LLM

In [25]:
def faz_previsao(entrada, solution_length = 6, model = modelo_llm):

    model.eval()
    entrada = torch.tensor(tokenizador(entrada))
    solution = []

    for i in range(solution_length):
        saida = model(entrada)
        predicted = saida.logits[-1].argmax()
        entrada = torch.cat((entrada, predicted.unsqueeze(0)), dim = 0)
        solution.append(predicted.cpu().item())

    return tokenizador.decode(solution)

# Testando o LLM

In [26]:
faz_previsao('3 + 5 ', solution_length=2)

'0 8'

In [27]:
faz_previsao('3 + 5 =', solution_length= 2)

'0 8'

In [28]:
faz_previsao('8 + 1 =', solution_length=2)

'0 9'

# Web App Para Deploy do LLM

In [29]:
def retorna_previsao(entrada):
    return faz_previsao(entrada, solution_length=2)

In [30]:
webapp = gr.Interface(fn = retorna_previsao,
                      inputs = [gr.Textbox(label= 'Dados de Entrada',
                                           lines = 1,
                                           info = "Os dados devem estar na forma: '1 + 2 =' com um único espaço entre cada caractere e apenas números de um dígito são permitidos.")],
                      outputs = [gr.Textbox(label= 'Resultado (Previsãao do Modelo)', lines = 1)],
                      title = 'Deploy do LLM',
                      description = 'Digite os dados de entrada e clique no botão Submit para o modelo fazer a previsão.',
                      examples = ['5 + 3 =', '2 + 9 ='])

In [31]:
webapp.launch(share= True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7b394b34b75abf8590.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
